In [1]:
file = open(r"C:\Users\99bis\Downloads\ahistoryofdepression.txt", encoding='utf-8')
text = file.read()

In [2]:
text

'When is sorrow sickness? So begins Jonathan Sadowsky’s The Empire of Depression, a history riven with professional turf wars around where to draw the line between normal sadness and something more serious — now, across much of the world, called depression. He argues against reductionism and dogma. Instead of getting stuck in old disagreements about whether depression is caused by a chemical imbalance or by social inequality, Sadowsky urges that depression can be psychological, biological and social, just as it can be a real illness even if it is cultural.\n\nGiven that the World Health Organization names depression as a major contributor to the global burden of disease, tracing its history is a significant task. And it is an important one, given the mental-health crisis attending the COVID-19 pandemic. It is no mean feat to characterize something that has ever-shifting and contested boundaries dependent on time and place. Sadowsky, a historian of medicine, offers three possible reason

In [3]:
# Function to load Lottie animations
def load_lottie_url(url: str):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('popular')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\99bis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\99bis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\99bis\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\99bis\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\99bis\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\99bis\AppData\Roaming\nltk

True

In [5]:
#Step 2- Extract the important words(keywords) from the text article that can be used to create MCQ using PKE (Python Keyword Extraction)

import pke
from nltk.corpus import stopwords #Stopwords are the words that we need to avoid while considering keyword extraction
import string

# Downloading NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Define the function to extract important words
def getImportantWords(art): 
    extractor = pke.unsupervised.MultipartiteRank() # Using the Multipartite Unsupervised Extractor as our extractor
    extractor.load_document(input=art, language='en')
    pos = {'PROPN'} # We are only considering proper nouns as valid candidates for our keywords
    stops = list(string.punctuation) # Stoplist contains the words to be avoided
    stops += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-'] # These stand for the brackets as in lrb=left round bracket="(" and so on
    stops += stopwords.words('english')
    extractor.candidate_selection(pos=pos) # Sets the candidate selection criteria, as in, which should be considered and which should be avoided
    extractor.candidate_weighting() # Sets the preference criteria for the candidates
    keyphrases = extractor.get_n_best(n=25) # Gets the 25 best candidates according to the criteria set
    result = [keyphrase[0] for keyphrase in keyphrases]
    return result
impWords=getImportantWords(text) #Get the important words (keywords) from the text article using the above function
print(impWords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\99bis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\99bis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['jonathan sadowsky', 'depression', 'empire', 'emily martin', 'bipolar expeditions', 'sadowsky', 'panic diaries', 'jackie orr', 'united states', 'black power', 'ian marsh', 'north america', 'britain', 'africans', 'sigmund freud', 'prozac', 'middle ages', 'renaissance europe', 'world health organization', 'imperial bedlam']


In [6]:
#Step 3- Split the whole text article into an array/list of individual sentences. This will help us fetch the sentences related to the keywords easily

from nltk.tokenize import sent_tokenize
def splitTextToSents(art):
    s=[sent_tokenize(art)]
    s=[y for x in s for y in x]
    s=[sent.strip() for sent in s if len(sent)>15] #Removes all the sentences that have length less than 15 so that we can ensure that our questions have enough length for context
    return s
sents=splitTextToSents(text) #Achieve a well splitted set of sentences from the text article
print(sents)

['When is sorrow sickness?', 'So begins Jonathan Sadowsky’s The Empire of Depression, a history riven with professional turf wars around where to draw the line between normal sadness and something more serious — now, across much of the world, called depression.', 'He argues against reductionism and dogma.', 'Instead of getting stuck in old disagreements about whether depression is caused by a chemical imbalance or by social inequality, Sadowsky urges that depression can be psychological, biological and social, just as it can be a real illness even if it is cultural.', 'Given that the World Health Organization names depression as a major contributor to the global burden of disease, tracing its history is a significant task.', 'And it is an important one, given the mental-health crisis attending the COVID-19 pandemic.', 'It is no mean feat to characterize something that has ever-shifting and contested boundaries dependent on time and place.', 'Sadowsky, a historian of medicine, offers th

In [7]:
#Step 4- Map the sentences which contain the keywords to the related keywords so that we can easily lookup the sentences related to the keywords

from flashtext import KeywordProcessor
def mapSents(impWords,sents):
    processor=KeywordProcessor() #Using keyword processor as our processor for this task
    keySents={}
    for word in impWords:
        keySents[word]=[]
        processor.add_keyword(word) #Adds key word to the processor
    for sent in sents:
        found=processor.extract_keywords(sent) #Extract the keywords in the sentence
        for each in found:
            keySents[each].append(sent) #For each keyword found, map the sentence to the keyword
    for key in keySents.keys():
        temp=keySents[key]
        temp=sorted(temp,key=len,reverse=True) #Sort the sentences according to their decreasing length in order to ensure the quality of question for the MCQ 
        keySents[key]=temp
    return keySents
mappedSents=mapSents(impWords,sents) #Achieve the sentences that contain the keywords and map those sentences to the keywords using this function
print(mappedSents)

{'jonathan sadowsky': ['So begins Jonathan Sadowsky’s The Empire of Depression, a history riven with professional turf wars around where to draw the line between normal sadness and something more serious — now, across much of the world, called depression.'], 'depression': ['The power to say who’s rational and who isn’t, and to detain people or treat them without consent, is perhaps the starkest reminder of why treating depression is not just like administering insulin for diabetes, and of why stigma looms large despite (or because of) the understandable appeals to biochemistry.', 'Sadowsky, a historian of medicine, offers three possible reasons for the current boom in diagnoses: that there really is more depression; that the amount is the same but we’re better at detecting it; or that emotional states not previously seen as illness are now being labelled as such.', 'Instead of getting stuck in old disagreements about whether depression is caused by a chemical imbalance or by social ine

In [8]:
# pip install pywsd

In [9]:
#Step 5- Get the sense of the word. In order to attain a quality set of distractors we need to get the right sense of the keyword. This is explained in detail in the seperate alogrithm documentation

from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
def getWordSense(sent,word):
    word=word.lower() 
    if len(word.split())>0: #Splits the word with underscores(_) instead of spaces if there are multiple words
        word=word.replace(" ","_")
    synsets=wn.synsets(word,'n') #Sysnets from Google are invoked
    if synsets:
        wup=max_similarity(sent,word,'wup',pos='n')
        adapted_lesk_output = adapted_lesk(sent, word, pos='n')
        lowest_index=min(synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None
print("fin")

Warming up PyWSD (takes ~10 secs)... 

fin


took 6.59632420539856 secs.


In [10]:
#Step 6- Get distractor from WordNet. These distractors work on the basis of hypernym and hyponym explained in detail in the documentation.

def getDistractors(syn,word):
    dists=[]
    word=word.lower()
    actword=word
    if len(word.split())>0: #Splits the word with underscores(_) instead of spaces if there are multiple words
        word.replace(" ","_")
    hypernym = syn.hypernyms() #Gets the hypernyms of the word
    if len(hypernym)==0: #If there are no hypernyms for the current word, we simple return the empty list of distractors
        return dists
    for each in hypernym[0].hyponyms(): #Other wise we find the relevant hyponyms for the hypernyms
        name=each.lemmas()[0].name()
        if(name==actword):
            continue
        name=name.replace("_"," ")
        name=" ".join(w.capitalize() for w in name.split())
        if name is not None and name not in dists: #If the word is not already present in the list and is different from he actial word
            dists.append(name)
    return dists
print("fin")

fin


In [11]:
#Step 7- The primary goal of this step is to take our MCQ quality one step further. The WordNet might some times fail to produce a hypernym for some words.
#In that case the ConcepNet comes into play as they help achieve our distractors when there are no hypernyms present for it in the WordNet. More about this is discussed
#in the algorithm documentation.

import requests
import json
def getDistractors2(word):
    word=word.lower()
    actword=word
    if len(word.split())>0: #Splits the word with underscores(_) instead of spaces if there are multiple words
        word=word.replace(" ","_")
    dists=[]
    url= "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word) #To get ditractors from ConceptNet's API
    obj=requests.get(url).json()
    for edge in obj['edges']:
        link=edge['end']['term']
        url2="http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2=requests.get(url2).json()
        for edge in obj2['edges']:
            word2=edge['start']['label']
            if word2 not in dists and actword.lower() not in word2.lower(): #If the word is not already present in the list and is different from he actial word
                dists.append(word2)
    return dists


In [12]:
# Step 8 (continued): Find and map the distractors to the keywords
mappedDists = {}
correctAnswers = {}  # Dictionary to store correct answers

for each in mappedSents:
    wordsense = getWordSense(mappedSents[each][0], each)
    if wordsense:
        dists = getDistractors(wordsense, each)
        if len(dists) == 0:
            dists = getDistractors2(each)
        if len(dists) != 0:
            mappedDists[each] = dists
            correctAnswers[each] = each.capitalize()  # Store correct answer
    else:
        dists = getDistractors2(each)
        if len(dists) > 0:
            mappedDists[each] = dists
            correctAnswers[each] = each.capitalize()

In [13]:
import re 

In [14]:
# Step 9- The final step is to present our MCQ in a nice and readable manner.

num_mcqs = int(input("Enter the number of questions you want : "))  # control the number of MCQs generated

print("**************************************        Multiple Choice Questions        *******************************")
print()
import re
import random
iterator = 1  # To keep the count of the questions
for each in mappedDists:
    if iterator > num_mcqs:
        break  # exit the loop if the desired number of MCQs has been reached
    sent = mappedSents[each][0]
    p = re.compile(each, re.IGNORECASE)  # Converts into regular expression for pattern matching
    op = p.sub("________", sent)  # Replaces the keyword with underscores(blanks)
    correct_answer = each.capitalize()  # The correct answer
    print("Question %s-> "%(iterator), op)  # Prints the question along with a question number
    options = [each.capitalize()] + mappedDists[each]  # Capitalizes the options
    options = options[:4]  # Selects only 4 options
    opts = ['a', 'b', 'c', 'd']
    random.shuffle(options)  # Shuffle the options so that order is not always same
    for i, ch in enumerate(options):
        print("\t", opts[i], ") ", ch)  # Print the options
    print("Correct Answer: ", correct_answer)  # Print the correct answer
    print()
    iterator += 1  # Increase the counter

**************************************        Multiple Choice Questions        *******************************

Question 1->  The power to say who’s rational and who isn’t, and to detain people or treat them without consent, is perhaps the starkest reminder of why treating ________ is not just like administering insulin for diabetes, and of why stigma looms large despite (or because of) the understandable appeals to biochemistry.
	 a )  Aquifer
	 b )  Beach
	 c )  Cave
	 d )  Depression
Correct Answer:  Depression

Question 2->  It is alarming, then, that apart from using “________” as an analogy for the global dominance of Western psychiatry in interpreting distress, Sadowsky devotes little attention here to power and politics — especially given his previous work on colonialism (in the 1999 book Imperial Bedlam).
	 a )  Archduchy
	 b )  Duchy
	 c )  Barony
	 d )  Empire
Correct Answer:  Empire

Question 3->  In fact, many forms of resistance have been deemed symptoms of mental illness